# ADK (Agent Development Kit) Tutorial - How to Build Agentic Systems with Gemini Models

## Prerequisites

This notebook demonstrates using Google's ADK (Agent Development Kit) with Gemini models. You have two options for authentication:

### Option 1: Google AI Studio (Gemini API Key)

For personal projects, experiments, or if you don't have a Google Cloud account:

1. Go to [Google AI Studio](https://makersuite.google.com/)
2. Create a free account or sign in with your Google account
3. Navigate to "Get API Key" in the top right or via settings
4. Generate a new API key
5. Create a `.env` file in the same directory as this notebook with:
   ```
   GOOGLE_API_KEY=your_api_key_here
   USE_VERTEX_AI=False
   ```

### Option 2: Vertex AI (Google Cloud)

For production environments, enterprise use, or when you need additional features:

1. Create a [Google Cloud account](https://cloud.google.com/) if you don't have one
2. Create a new project or select an existing one
3. Enable the Vertex AI API in your project
4. Set up authentication:
   - Create a service account and download the JSON key, or
   - Use Google Cloud SDK for local development (`gcloud auth application-default login`)
5. Create a `.env` file with:
   ```
   GOOGLE_CLOUD_PROJECT=your_project_id
   GOOGLE_CLOUD_LOCATION=us-central1  # or your preferred region
   USE_VERTEX_AI=True
   ```

### Which Option Should I Use?

- **Use Google AI Studio API Key (Option 1) if:**
  - You're just getting started/experimenting
  - You don't have a Google Cloud account
  - You want a simpler setup process
  - You're working on a personal project

- **Use Vertex AI (Option 2) if:**
  - You need enterprise-grade security and compliance
  - You require higher rate limits
  - You're building for production deployment
  - You need advanced Vertex AI features

### Required Python Packages

Install the required packages:

```bash
pip install -r requirements.txt
```

This notebook automatically detects your configuration based on the `USE_VERTEX_AI` environment variable.

In [1]:
import os
import asyncio
import warnings
import logging
from dotenv import load_dotenv
import vertexai
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types


### Constants

In [2]:
# Clean setup
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)
load_dotenv()

# Check if running in Vertex AI or AI Studio
USE_VERTEX_AI = os.getenv("USE_VERTEX_AI", "False").lower() == "True"
MODEL_GEMINI_FLASH = "gemini-2.0-flash"

if USE_VERTEX_AI:
    # Vertex AI setup
    from vertexai import init as vertexai_init
    from vertexai.generative_models import GenerativeModel

    PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]
    LOCATION = os.environ["GOOGLE_CLOUD_LOCATION"]
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

    vertexai_init(project=PROJECT_ID, location=LOCATION)
    # model = GenerativeModel(MODEL_GEMINI_FLASH)
else:
    # AI Studio setup
    import google.generativeai as genai

    API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY environment variable is not set.")

    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel("gemini-2.0-flash")

### A Vanilla Agent

In [3]:
#  A Vanilla AI Agent
simple_agent = Agent(
    name="helpful_assistant",
    model=MODEL_GEMINI_FLASH,
    description="A friendly AI that answers questions",
    instruction="You are helpful and conversational. Keep answers concise.",
    tools=[]  # No tools yet, just conversation
)

# Setup session (think of it as conversation memory)
session_service = InMemorySessionService()
APP_NAME = "my_first_app"
USER_ID = "user_123"
SESSION_ID = "chat_001"

# Create the session
await session_service.create_session(
    app_name=APP_NAME, 
    user_id=USER_ID, 
    session_id=SESSION_ID
)

# Create runner (this handles the conversation)
runner = Runner(
    agent=simple_agent, 
    app_name=APP_NAME, 
    session_service=session_service
)

In [4]:
# Test your agent:
async def chat_with_agent(message):
    """Send a message and get response"""
    print(f"🧑 You: {message}")
    
    content = types.Content(
        role="user", 
        parts=[types.Part(text=message)]
    )
    
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=content
    ):
        if event.is_final_response() and event.content:
            response = event.content.parts[0].text
            print(f"🤖 Agent: {response}")
            break

# Try it out!
await chat_with_agent("Hello! What all things you can do for me?")

🧑 You: Hello! What all things you can do for me?
🤖 Agent: I can answer questions, provide summaries, generate creative content, and have conversations on a variety of topics.



### Adding Tool to Vanilla Agent

In [5]:
from google.adk.tools import google_search

# Create an agent with search powers
search_agent = Agent(
    name="search_assistant",
    model=MODEL_GEMINI_FLASH,
    description="AI assistant that can search the internet",
    instruction="""You can search the internet for current information. 
    When users ask about recent events or current info, use google_search.""",
    tools=[google_search]  # This is the only new line!
)

# Create new runner for search agent
search_runner = Runner(
    agent=search_agent,
    app_name=APP_NAME,
    session_service=session_service
)

async def search_chat(message):
    """Chat with search-enabled agent"""
    print(f"🧑 You: {message}")
    print("🔍 Searching...")
    
    content = types.Content(role="user", parts=[types.Part(text=message)])
    
    async for event in search_runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,   # Different session
        new_message=content
    ):
        if event.is_final_response() and event.content:
            response = event.content.parts[0].text.replace("**", "")
            print(f"🤖 Agent: {response}")
            break

# Try searching for current events
await search_chat("What's the latest breakthrough in AI research?")

🧑 You: What's the latest breakthrough in AI research?
🔍 Searching...
🤖 Agent: AI research is rapidly advancing across many fields. Here are some of the latest breakthroughs:

*   AI-Driven Scientific Discovery: AI is accelerating breakthroughs in various scientific fields, including supercomputing, weather forecasting, drug discovery, and sustainable materials. For example, Microsoft Research developed AI2BMD, an AI-driven protein simulation system that helps researchers explore biomolecular science problems with unprecedented speed and precision.
*   Robotics and Haptics: Researchers have created a revolutionary robotic skin made from a flexible, low-cost gel material that brings machines closer to human-like touch by allowing them to feel heat, pain, and pressure. Additionally, a four-legged AI robot has been developed that can play badminton with humans, showcasing the potential for human-robot collaboration in sports and training.
*   Quantum Computing Enhancement: Quantum computin

### Vision Agent with Tool Use - Understanding Images

In [6]:
import base64
from pathlib import Path

def encode_image_to_base64(image_path):
    """Convert image to base64 for AI processing"""
    return base64.b64encode(Path(image_path).read_bytes()).decode()

# Create vision-enabled agent
vision_agent = Agent(
    name="vision_assistant",
    model=MODEL_GEMINI_FLASH,
    description="AI that can see and understand images",
    instruction="""You can analyze images and answer questions about them. 
    Describe what you see clearly and answer any questions about the image.""",
    tools=[google_search]  # Can also search if needed
)

vision_runner = Runner(
    agent=vision_agent,
    app_name=APP_NAME,
    session_service=session_service
)

In [7]:
async def chat_with_image(message, image_path=None, image_base64=None):
    """Send message with optional image"""
    print(f"🧑 You: {message}")

    # Create message parts
    parts = [types.Part(text=message)]
    
    # Add image if provided
    if image_path:
        data = Path(image_path).read_bytes()
        mime_type = "image/jpeg" if image_path.endswith(('.jpg', '.jpeg')) else "image/png"
        parts.append(types.Part.from_bytes(data=data, mime_type=mime_type))
    elif image_base64:
        data = base64.b64decode(image_base64)
        parts.append(types.Part.from_bytes(data=data, mime_type="image/jpeg"))

    content = types.Content(role="user", parts=parts)

    async for event in vision_runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=content
    ):
        if event.is_final_response() and event.content:
            response = event.content.parts[0].text.replace("**", "")
            print(f"🤖 Agent: {response}")
            break
        
await chat_with_image("What do you see in this image?", image_path="images/image.jpg")

🧑 You: What do you see in this image?
🤖 Agent: The image shows two iPhones, presumably the iPhone 14 Pro or a similar model, in what appears to be a gold or bronze color. The phone on the left displays the rear side, featuring a triple-lens camera system and the Apple logo. The phone on the right showcases the front side with the display on, which has a dark abstract wallpaper with orange accents. The overall presentation of the image suggests it is for promotional or display purposes.



### Multi Modal - Multi Agent System with Custom Tool Use

In [10]:
import time
from pathlib import Path
from datetime import date
from google.adk.agents import Agent, SequentialAgent

USER_ID = "user_001"
session_service = InMemorySessionService()

def years_since(year: int) -> int:
    """Calculate how many years have passed since a given year"""
    today = date.today()
    years_passed = today.year - year
    print(f"🔧 Tool used: {years_passed} years have passed since {year}")
    return years_passed

print("✅ Custom tool added!")

async def chat_with_expert(message, image_path=None):
    """Chat function to interact with the expert system"""
    
    session_id = f"multimodal_{USER_ID}_{int(time.time())}"
    await session_service.create_session(
        app_name="entertainment_app",
        user_id=USER_ID,
        session_id=session_id
    )

    print(f"🧑 You: {message}")
    parts = [types.Part(text=message)]

    if image_path:
        try:
            image_file = Path(image_path)
            if image_file.exists():
                print("📸 Processing image...")
                data = image_file.read_bytes()
                if image_path.lower().endswith(('.jpg', '.jpeg')):
                    mime_type = "image/jpeg"
                elif image_path.lower().endswith('.png'):
                    mime_type = "image/png"
                elif image_path.lower().endswith('.webp'):
                    mime_type = "image/webp"
                else:
                    mime_type = "image/jpeg"
                parts.append(types.Part.from_bytes(data=data, mime_type=mime_type))
            else:
                print(f"❌ Image file not found: {image_path}")
        except Exception as e:
            print(f"❌ Error loading image: {e}")
    
    content = types.Content(role="user", parts=parts)

    try:
        async for event in expert_runner.run_async(
            user_id=USER_ID,
            session_id=session_id,
            new_message=content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                response = event.content.parts[0].text.replace("**", "")
                print(f"🎬 Pop Culture Expert: {response}")
    except Exception as e:
        print(f"❌ Error: {e}")

# === Step 2: Define Sub-Agents ===

image_identifier = Agent(
    name="image_identifier",
    model=MODEL_GEMINI_FLASH,
    description="Identify content from image",
    instruction="Identify the game, movie, or TV show in the image. Return only the title and when was it released using google_search tool",
    tools=[google_search]
)

age_calculator = Agent(
    name="age_calculator",
    model=MODEL_GEMINI_FLASH,
    description="Calculate how long ago the content was released years_since tool",
    instruction="Take the title, the date of release and calculate its release year and age. Format: 'Released X years ago in YYYY'",
    tools=[years_since]
)

news_searcher = Agent(
    name="news_searcher",
    model=MODEL_GEMINI_FLASH,
    description="Search for recent news about the title",
    instruction="Use the title to find 2-3 recent news updates.",
    tools=[google_search]
)

# === Step 3: Define Fixed Sequential Agent ===

class SequentialEntertainmentExpert(SequentialAgent):
    async def run(self, user_id, session_id, new_message):
        # Step 1: Identify title
        step1 = await self.sub_agents[0].run(user_id=user_id, session_id=session_id, new_message=new_message)
        title = step1.final_response.text.strip()
        print(f"🔍 Detected title: {title}")

        # Step 2: Calculate age
        age_msg = types.Content(role="user", parts=[types.Part(text=title)])
        step2 = await self.sub_agents[1].run(user_id=user_id, session_id=session_id, new_message=age_msg)
        age_info = step2.final_response.text.strip()
        print(f"📅 Age Info: {age_info}")

        # Step 3: Get news
        news_msg = types.Content(role="user", parts=[types.Part(text=title)])
        step3 = await self.sub_agents[2].run(user_id=user_id, session_id=session_id, new_message=news_msg)
        news_info = step3.final_response.text.strip()
        print(f"🗞️ News Info: {news_info}")

        # Combine all
        summary = f"This is **{title}**. {age_info}. Recent news: {news_info}"
        return types.Response(content=types.Content(role="assistant", parts=[types.Part(text=summary)]))

# === Step 4: Create Runner ===

def create_sequential_multimodal_fixed():
    fixed_sequential_agent = SequentialEntertainmentExpert(
        name="sequential_entertainment_expert_fixed",
        description="Sequential agent with explicit chaining",
        sub_agents=[image_identifier, age_calculator, news_searcher]
    )

    return Runner(
        agent=fixed_sequential_agent,
        app_name="entertainment_app",
        session_service=session_service
    )

# Assign fixed runner globally
expert_runner = create_sequential_multimodal_fixed()

# === Step 5: Run Test ===
await chat_with_expert(
        "What game is this, how old is it, and what's the latest news?",
        "images/mario.jpg"  # Ensure this image exists locally
    )

✅ Custom tool added!
🧑 You: What game is this, how old is it, and what's the latest news?
📸 Processing image...
🎬 Pop Culture Expert: The game in the image is Super Mario Bros., which was released on September 13, 1985, making it almost 40 years old. While the exact North American release date is disputed, Nintendo officially recognizes it as October 18, 1985.

Here's some of the latest news in the world of Super Mario:
*   New Games: There are several new Mario games either recently released or coming soon, including Donkey Kong Bananza, Super Mario Party Jamboree, Mario Kart World, and Mario & Luigi: Brothership.
*   Super Mario Bros. Wonder: Released in October 2023 for the Nintendo Switch, Super Mario Bros. Wonder has sold over 16 million copies as of March 31, 2025.
*   New Super Mario Bros. U Deluxe: A Switch 2 update of New Super Mario Bros. U Deluxe has been released.

🔧 Tool used: 40 years have passed since 1985
🎬 Pop Culture Expert: Released 40 years ago in 1985.

🎬 Pop Cultu